# tf.function and autograph
* [source site](https://www.tensorflow.org/beta/tutorials/eager/tf_function)

In [5]:
import tensorflow as tf
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

In [6]:
assert_raises(print('d'))

d


In [7]:
# A function is like an op

@tf.function
def tf_add(a, b):
    return a + b

tf_add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [8]:
tf_add(3,3)

<tf.Tensor: id=20, shape=(), dtype=int32, numpy=6>

In [9]:
def add(a, b):
    return a + b
add(3,3)

6

In [10]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = tf_add(v, 3.0)
tape.gradient(result, v)

<tf.Tensor: id=45, shape=(), dtype=float32, numpy=1.0>

In [11]:
# You can use functions inside functions

@tf.function
def dense_layer(x, w, b):
    return tf_add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=72, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

### tensor function can be polymorphic.

In [12]:
# Functions are polymorphic

@tf.function
def double(a):
  print("Tracing with", a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [13]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string)) # receive only string variable

Obtaining concrete trace
Tracing with Tensor("a:0", dtype=string)


In [14]:
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))

Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [15]:
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1)) # data type error

Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_103 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_103]


In [16]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(tf.equal(x % 2, 0), x // 2, 3 * x + 1) # where x is even number? if is true return quotient(몫) from
                                                           # diving 2, or not reture 3 * x + 1.

print(next_collatz(tf.constant([1, 2])))


Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)


In [17]:
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=130, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name=None),))


In [18]:
tf.where(tf.equal(2 % 2, 0), 2 // 2, 3 * 2 + 1)

<tf.Tensor: id=137, shape=(), dtype=int32, numpy=1>

In [19]:
tf.where(True, 2, 7)

<tf.Tensor: id=142, shape=(), dtype=int32, numpy=2>

In [20]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()

train(num_steps=10)
train(num_steps=20)

Tracing with num_steps = 10
Tracing with num_steps = 20


In [21]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


### Side effects in tf.function

In [22]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

f(1)
f(1)
f(2)

Traced with 1
Executed with 1
Executed with 1
Traced with 2
Executed with 2


In [23]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

W0716 14:46:02.886202 139722710042368 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0716 14:46:02.887730 139722710042368 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32
W0716 14:46:02.888507 139722710042368 backprop.py:842] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.int32


Python side effect
Python side effect
Python side effect


In [24]:
len(external_list) == 3

True

In [25]:
external_list[0].numpy() == 1

True

In [26]:
external_list[0]

<tf.Tensor: id=309, shape=(), dtype=int32, numpy=1>

In [27]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


In [28]:

x = iter([0, 1, 2, 3])
l = [0, 1, 2, 3]
for i in l:
    print(i)
    print(next(x))
   # print(next(i))

0
0
1
1
2
2
3
3


In [29]:
tf_var = tf.Variable(0)
print(tf_var)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>


In [30]:
tf_var.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=1>

In [31]:
print(tf_var)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>


### python iterator don't work with tf.function.
### so you have to use tf.data.Dataset.

In [32]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(
      f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # Some dummy computation.
    return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph


In [33]:
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

W0716 14:46:03.138170 139728635369280 deprecation.py:323] From /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph


In [34]:
test_data = [(1,2)] *2
for x, y in test_data:
    print(x)
    print(y)

1
2
1
2


In [35]:
small_tf_data = tf.data.Dataset.from_generator(lambda: small_data, (tf.int32, tf.int32))

In [36]:
measure_graph_size(train, small_tf_data)

train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 4 nodes in its graph


In [37]:
for x, y in small_tf_data:
    print(x)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)


In [38]:
for x, y in test_data:
    print(x)

1
1


## Automatic control dependencies

In [39]:
# Automatic control dependencies

a = tf.Variable(4.0)
b = tf.Variable(6.0)


@tf.function
def f(x, y):
    a.assign(y * b) # 2.0 x 6.0 = 12.0 -> a
    b.assign_add(x * a) # 1.0 x 12.0 + 6.0 = 18.0
    return a + b 

f(1.0, 2.0)  # 30.0

<tf.Tensor: id=529, shape=(), dtype=float32, numpy=30.0>

In [40]:
v = None

print(v)
@tf.function
def f(x):
    if v is None:
         v = tf.Variable(1.0)
    print(v)
    return v.assign_add(x)

with assert_raises(ValueError):
    f(1.0)

None
Got unexpected exception 
  <class 'AttributeError'>: in converted code:

    <ipython-input-40-cd10815703a4>:9 f  *
        return v.assign_add(x)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:329 converted_call
        f = getattr(owner, f)

    AttributeError: 'Undefined' object has no attribute 'assign_add'



In [41]:
# Non-ambiguous code is ok though

v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # already v is converted 2.0, so result will be 4.0

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [42]:
print(f(1.0))  # from v=4.0
print(f(2.0))  # from v=5.0

tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)


In [43]:
# You can also create variables inside a tf.function as long as we can prove
# that those variables are created only the first time the function is executed.

class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    print(obj.v)
    return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

<tf.Variable 'Variable:0' shape=() dtype=float32>
<tf.Variable 'Variable:0' shape=() dtype=float32>
tf.Tensor(2.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32>
tf.Tensor(4.0, shape=(), dtype=float32)


In [44]:
# Variable initializers can depend on function arguments and on values of other
# variables. We can figure out the right initialization order using the same
# method we use to generate control dependencies.

state = []
@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 * x)) # 1.0 -> state[0] = 2.0
        state.append(tf.Variable(state[0] * 3.0)) # state[1] = 6.0
    return state[0] * x * state[1] #  (2.0 * x * 6.0)

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0))) # state has values, so doesn't append any values.
state

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>]

In [45]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        tf.print(tf.reduce_sum(x))
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))


[0.0684449673 0.652370334 0.359579086 0.630169153 0.4163481]
2.12691164
[0.0683382899 0.573263526 0.344843209 0.55816865 0.393849462]
1.93846309
[0.0682321042 0.517751932 0.331794322 0.506617367 0.374674052]
1.79906988
[0.0681264177 0.475963056 0.320132107 0.467305422 0.358073264]
1.68960023
[0.0680212155 0.443004757 0.309626341 0.436019599 0.343515784]
1.60018766
[0.067916505 0.416132 0.300097108 0.410339683 0.33061251]
1.52509785
[0.0678122714 0.393666863 0.291401476 0.388761073 0.319071025]
1.46071267
[0.0677085146 0.374517083 0.283424199 0.37029168 0.308666676]
1.40460813
[0.0676052347 0.357936412 0.27607128 0.354246825 0.299223632]
1.35508347
[0.0675024241 0.343395084 0.269265056 0.340136439 0.290601969]
1.31090093
[0.0674000829 0.330504984 0.262940824 0.327599198 0.282688767]
1.2711339
[0.0672982112 0.318974435 0.257044107 0.316361904 0.275391757]
1.23507047
[0.0671967939 0.308579296 0.25152871 0.306213617 0.268634707]
1.20215309
[0.0670958385 0.299144059 0.246355131 0.296988577 

<tf.Tensor: id=775, shape=(5,), dtype=float32, numpy=
array([0.06630425, 0.24582867, 0.21398729, 0.24463177, 0.22420436],
      dtype=float32)>

In [46]:
# If you're curious you can inspect the code autograph generates.
# It feels like reading assembly language, though.
# note that it omits @tf.function

def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    tf.print(tf.reduce_sum(x))
    x = tf.tanh(x)
  return x

print(tf.autograph.to_code(f))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()

  def loop_test(x_1):
    return ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None) > 1

  def loop_body(x_1):
    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None),), None)
    x_1 = ag__.converted_call('tanh', tf, ag__.ConversionOptions(recursive=True, force_conversion=False, optional_features=(), internal_convert_user_code=True), (x_1,), None)
    return x

### Autograph and condition

In [47]:
def test_tf_cond(f, *args):
    g = f.get_concrete_function(*args).graph
    if any(node.name == 'cond' for node in g.as_graph_def().node):
        print("{}({}) uses tf.cond.".format(
            f.__name__, ', '.join(map(str, args))))
    else:
        print("{}({}) executes normally.".format(
          f.__name__, ', '.join(map(str, args))))

In [48]:
@tf.function
def hyperparam_cond(x, training=True):
    if training:
        x = tf.nn.dropout(x, rate=0.5)
    return x

@tf.function
def maybe_tensor_cond(x):
    if x < 0:
        x = -x
    return x

test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))
test_tf_cond(maybe_tensor_cond, tf.constant(-1))
test_tf_cond(maybe_tensor_cond, -1) #why?

hyperparam_cond(tf.Tensor([1.], shape=(1,), dtype=float32)) executes normally.
maybe_tensor_cond(tf.Tensor(-1, shape=(), dtype=int32)) uses tf.cond.
maybe_tensor_cond(-1) executes normally.


In [50]:
test_tf_cond(maybe_tensor_cond, tf.constant(1))
test_tf_cond(maybe_tensor_cond, 1)

maybe_tensor_cond(tf.Tensor(1, shape=(), dtype=int32)) uses tf.cond.
maybe_tensor_cond(1) executes normally.


In [53]:
@tf.function
def f():
    x = tf.constant(0)
    if tf.constant(True):
        x = x + 1
        print("Tracing `then` branch")
    else:
        x = x - 1
        print("Tracing `else` branch") # if you use tf.function they trace both branch. Thus, if you comment @tf.function 
                                       # this function ony print then branch.
    return x

f()

Tracing `then` branch
Tracing `else` branch


<tf.Tensor: id=886, shape=(), dtype=int32, numpy=1>

In [62]:
@tf.function
def f():
    if tf.constant(True):
        x = tf.ones([3, 3])
    return x

# Throws an error because both branches need to define `x`.
with assert_raises(ValueError):
    f()

Caught expected exception 
  <class 'ValueError'>: in converted code:

    <ipython-input-62-810946e9b87f>:3 f  *
        if tf.constant(True):
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/autograph/operators/control_flow.py:439 if_stmt
        return tf_if_stmt(cond, body, orelse, get_state, set_state)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/autograph/operators/control_flow.py:456 tf_if_stmt
        outputs, final_state = control_flow_ops.cond(cond, body, orelse)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507 new_func
        return func(*args, **kwargs)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:1147 cond
        return cond_v2.cond_v2(pred, true_fn, false_fn, name)
    /home/hyunsu/anaconda3/envs/tf20_py36/lib/python3.6/site-packages/tensorflow/python/ops/cond_v

* from AutoGraph and loops
* it's how to understand codes on current my stage, so skipped.